In [ ]:
#!/usr/local/miniconda3/bin/python3
import os,sys
import numpy as np
import fortranformat
import h5py as h5
from tqdm import tqdm
from omfit_classes.omfit_eqdsk import OMFITeqdsk

f1040 = fortranformat.FortranRecordReader('1x,4e16.9')
f1041 = fortranformat.FortranRecordReader('1x,4i5')
f1060 = fortranformat.FortranRecordReader('A1,f8.3,9x,i5,11x,i5,1x,a3,1x,i3,1x,i3,1x,a3,1x,2i5')

In [35]:
def find_afile(shot, time):
    shot_dir = f'/projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_{shot}'
    files = os.listdir(shot_dir)
    times = []
    for f in files:
        times.append(int(f.split('.')[1]))

    times = np.array(times)

    a_file_time = times[np.argmin(np.abs(times - time))]

    return f'a{shot:06d}.{a_file_time:06d}'

In [37]:
shape_params = ['aminor', 'volume', 'elong', 'elongm', 'gapbot', 'gapin', 'gaptop', 'gapout',
                'ltri', 'rcntr', 'rcencm', 'rmidin', 'rmidout', 'rseps', 'utri', 'wmhd', 'zcntr', 'zm', 'zseps']

In [52]:
with h5.File('training_inputs.h5', 'r+') as h5_file:
    for shot in tqdm(h5_file.keys()):
        for time in h5_file[shot].keys():
            fn = find_afile(int(shot), int(1000.0 * float(time)))
            a_file = OMFITeqdsk(f'/projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_{shot}/'+fn)
            
            for shape in shape_params:
                h5_file[shot][time].create_dataset(shape, data=a_file[shape])

 47%|████▋     | 178/376 [03:09<05:21,  1.62s/it]bad time in aEQDSK file: /projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_33810/a033810.010030
bad time in aEQDSK file: /projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_33810/a033810.010130
bad time in aEQDSK file: /projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_33810/a033810.010230
bad time in aEQDSK file: /projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_33810/a033810.010320
bad time in aEQDSK file: /projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_33810/a033810.010430
bad time in aEQDSK file: /projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_33810/a033810.010530
bad time in aEQDSK file: /projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_33810/a033810.010630
bad time in aEQDSK file: /projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_33810/a033810.010720
bad time in aEQDSK file: /projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_33810/a033810.010830
bad time in aEQDSK file: /projects/EKOLEMEN/KSTAR_torbeam/a_files/shot_33810/a033810.010930
bad time in aEQDSK file: /proje

In [53]:
cmu_shape = ['elong', 'ltri', 'utri', 'rseps', 'zseps']
with h5.File('BO_shape.h5', 'w') as cmu_file:
    with h5.File('training_inputs.h5', 'r') as h5_file:
        for shot in tqdm(h5_file.keys()):
            shot_group = cmu_file.create_group(shot)
            for time in h5_file[shot].keys():
                time_group = shot_group.create_group(time)
                for shape in cmu_shape:
                    time_group.create_dataset(shape, data=h5_file[shot][time][shape])

100%|██████████| 376/376 [03:45<00:00,  1.67it/s]
